<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Relevant-imports" data-toc-modified-id="Relevant-imports-1">Relevant imports</a></span></li><li><span><a href="#Useful-datasets" data-toc-modified-id="Useful-datasets-2">Useful datasets</a></span></li><li><span><a href="#Food-deserts" data-toc-modified-id="Food-deserts-3">Food deserts</a></span><ul class="toc-item"><li><span><a href="#Grocery-stores-density" data-toc-modified-id="Grocery-stores-density-3.1">Grocery stores density</a></span></li><li><span><a href="#Diet-related-deaths" data-toc-modified-id="Diet-related-deaths-3.2">Diet-related deaths</a></span><ul class="toc-item"><li><span><a href="#Diabetes-related-deaths-(US-baseline-73.1-!)" data-toc-modified-id="Diabetes-related-deaths-(US-baseline-73.1-!)-3.2.1">Diabetes related deaths (US baseline 73.1 !)</a></span></li><li><span><a href="#Colorectal-Cancer-(US-baseline-17)" data-toc-modified-id="Colorectal-Cancer-(US-baseline-17)-3.2.2">Colorectal Cancer (US baseline 17)</a></span></li><li><span><a href="#Stroke-(US-baseline-42.2)" data-toc-modified-id="Stroke-(US-baseline-42.2)-3.2.3">Stroke (US baseline 42.2)</a></span></li></ul></li><li><span><a href="#Community-areas-by-ethnicity" data-toc-modified-id="Community-areas-by-ethnicity-3.3">Community areas by ethnicity</a></span></li></ul></li></ul></div>

# Relevant imports

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
import math
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter

import datetime

from autocorrect import Speller

from datetime import date as dt

import folium
from folium import plugins
from folium.plugins import HeatMap
#http://geopandas.org/install.html
import geopandas as gpd

import geopandas as gpd
from shapely.geometry import shape, Point

import requests
from pandas.io.json import json_normalize
import pandas_profiling

import warnings
warnings.filterwarnings('ignore')

from bokeh.io import curdoc, output_notebook
from bokeh.models import Slider, HoverTool
from bokeh.layouts import widgetbox, row, column

# Useful datasets

In [2]:
socioeconomic_factors = pd.read_csv("datasets/cleaned_socioeconomic_factors.csv")
inspections = pd.read_csv("datasets/cleaned_inspections.csv")
fp = "datasets/boundaries_processed.geojson"
boundaries_community_areas = gpd.read_file(fp)

In [3]:
socioeconomic_factors.head()

,Community Area,Community Area Name,Colorectal Cancer,Diabetes-related,Stroke (Cerebrovascular Disease),Below Poverty Level,Per Capita Income,Unemployment,Population
0,1,Rogers Park,25.3,77.1,33.7,22.7,23714,7.5,55500
1,2,West Ridge,17.3,60.5,34.7,15.1,21375,7.9,73723
2,3,Uptown,20.5,80.0,41.7,22.7,32355,7.7,56296
3,4,Lincoln Square,8.6,55.4,36.9,9.5,35503,6.8,41483
4,5,North Center,26.1,49.8,41.6,7.1,51615,4.5,34942


In [4]:
boundaries_community_areas.head()

,community,area_number,shape_area,geometry
0,DOUGLAS,35,4.600462e+07,"MULTIPOLYGON (((-87.60914 41.84469, -87.60915 ..."
1,OAKLAND,36,1.691396e+07,"MULTIPOLYGON (((-87.59215 41.81693, -87.59231 ..."
2,FULLER PARK,37,1.991670e+07,"MULTIPOLYGON (((-87.62880 41.80189, -87.62879 ..."
3,GRAND BOULEVARD,38,4.849250e+07,"MULTIPOLYGON (((-87.60671 41.81681, -87.60670 ..."
4,KENWOOD,39,2.907174e+07,"MULTIPOLYGON (((-87.59215 41.81693, -87.59215 ..."


In [5]:
inspections.head()

,Inspection ID,DBA Name,AKA Name,License #,Facility Type,Risk,Address,Inspection Date,Inspection Type,Results,Violations,Latitude,Longitude,Location,Community Area,Violation Numbers,Violation Comments,Reinspections,area_number
0,2345318,SUBWAY,SUBWAY,2529116,Restaurant,1,2620 N NARRAGANSETT AVE,2019-11-08,Canvass,Pass w/ Conditions,"3. MANAGEMENT, FOOD EMPLOYEE AND CONDITIONAL E...",41.927995,-87.785752,"41.92799528871574, -87.78575236468352",BELMONT CRAGIN,"[3, 5, 58]",[' 2-102.14(O) OBSERVED NO WRITTEN EMPLOYEE H...,1,19
1,2321166,SUBWAY,SUBWAY,2529116,Restaurant,1,2620 N NARRAGANSETT AVE,2019-11-06,Canvass,Fail,"3. MANAGEMENT, FOOD EMPLOYEE AND CONDITIONAL E...",41.927995,-87.785752,"41.92799528871574, -87.78575236468352",BELMONT CRAGIN,"[3, 5, 10, 22, 33, 57, 58]",[' OBSERVED NO WRITTEN EMPLOYEE HEALTH POLICY ...,0,19
2,2290717,PAPA JOHN'S PIZZA,PAPA JOHN'S PIZZA,1545897,Restaurant,2,2620 N NARRAGANSETT AVE,2019-05-30,Canvass,Pass w/ Conditions,"3. MANAGEMENT, FOOD EMPLOYEE AND CONDITIONAL E...",41.927995,-87.785752,"41.92799528871574, -87.78575236468352",BELMONT CRAGIN,"[3, 5]",[' OBSERVED NO EMPLOYEE HEALTH POLICY ON THE P...,0,19
3,2144892,SUBWAY,SUBWAY,2529116,Restaurant,1,2620 N NARRAGANSETT AVE,2018-02-13,Canvass,Pass,NaN,41.927995,-87.785752,"41.92799528871574, -87.78575236468352",BELMONT CRAGIN,NaN,NaN,1,19
4,2144588,SUBWAY,SUBWAY,2529116,Restaurant,1,2620 N NARRAGANSETT AVE,2018-02-06,Canvass,Fail,"16. FOOD PROTECTED DURING STORAGE, PREPARATION...",41.927995,-87.785752,"41.92799528871574, -87.78575236468352",BELMONT CRAGIN,"[16, 34, 42]",[' OBSERVED BLACK AND PINK MOLD LIKE SUBSTANCE...,0,19


In [6]:
inspections.loc[:,('Inspection Date')]=inspections.loc[:,('Inspection Date')].apply(pd.to_datetime)

In [7]:
inspections.dtypes

Inspection ID                  int64
DBA Name                      object
AKA Name                      object
License #                      int64
Facility Type                 object
Risk                           int64
Address                       object
Inspection Date       datetime64[ns]
Inspection Type               object
Results                       object
Violations                    object
Latitude                     float64
Longitude                    float64
Location                      object
Community Area                object
Violation Numbers             object
Violation Comments            object
Reinspections                  int64
area_number                    int64
dtype: object

# Food deserts

**Context:**

After a small research, we found out about a problem affecting public health in Chicago at a large scale called **Food deserts**. In a report by the Illinois Advisory Committee to the United States Commission on Civil Rights, it is reported that some of the community areas in Chicago are lacking supermarkets, grocery stores and healthy food in general. Restricted access to healthy foods lead to higher rates of chronic illness like diabetes, hypertension, or cardiovascular disease. [source](https://www.usccr.gov/pubs/docs/IL-FoodDeserts-2011.pdf)

Furthermore, it is stated in the report that these food desert neighborhoods are almost exclusively in African American neighborhoods making the issue a civil rights one beyond its public health dimension.

**In the next part we will try to see if we can visualize the food deserts using the food inspection dataset**

**Approach to the problem**

* We first try to look at establishments that represent healthy food providers (grocery store, farmer's market, fish market, meat market, etc). These establishments also have inspections and so we retrieve them from our dataset and plot their intensity by 1'000 habitants for the different community areas.

* We then try to visualize on a map the most common ethnicity accross all community areas as well as some socio economic factors.

* In the end we will show diet related disease rates by community area as well and try to see how it relates.

## Grocery stores density

In [8]:
#We care about grocery stores only
groceries_intensity = inspections[inspections['Facility Type']=='Market']
# We take distinct markets ranging from year 2015 to 2019
# This is because some establishment do not receive any inspection on a given year sometimes 
groceries_intensity = groceries_intensity[groceries_intensity['Inspection Date'].dt.year==2018]
groceries_intensity = groceries_intensity[['Community Area','DBA Name','area_number']].drop_duplicates().groupby(['Community Area','area_number'])['DBA Name'].agg({'nbr_markets' : len})
groceries_intensity.reset_index(inplace=True)
groceries_intensity.head(1)

,Community Area,area_number,nbr_markets
0,ALBANY PARK,14,25


We tried to compute intensity of markets by 10'000 habitant but it was not representative at all.
Because even if some areas are more dense in population the distribution of healthy food should be equal in terms of surface which is the essence of the problem as described in the [wikipedia page]    (https://en.wikipedia.org/wiki/Supermarket_shortage) which leads people to take long distance rides to have access to fresh food and furthermore the public transportation system is usually also bad for those areas.
We hence plot the density of markets by surface.

In [9]:
population_by_community = socioeconomic_factors[['Community Area','Population']]
groceries_intensity = groceries_intensity.merge(population_by_community,\
                              left_on='area_number', right_on='Community Area')

groceries_intensity = groceries_intensity.merge(boundaries_community_areas,\
                              left_on='area_number', right_on='area_number')



groceries_intensity['markets_by_surface'] = groceries_intensity['nbr_markets'].div(groceries_intensity['shape_area'].values,axis=0)
min_value, max_value = np.min(groceries_intensity['markets_by_surface']), np.max(groceries_intensity['markets_by_surface'])

groceries_intensity['markets_by_surface']=(groceries_intensity['markets_by_surface']-min_value)/max_value

groceries_intensity.drop(columns=['Community Area_y', 'community'], inplace=True)
groceries_intensity.rename(columns={'Community Area_x':'Community Area'}, inplace=True)

groceries_intensity.head()

,Community Area,area_number,nbr_markets,Population,shape_area,geometry,markets_by_surface
0,ALBANY PARK,14,25,51575,5.354223e+07,"MULTIPOLYGON (((-87.70404 41.97355, -87.70403 ...",0.654319
1,ARCHER HEIGHTS,57,1,13259,5.592251e+07,"MULTIPOLYGON (((-87.71437 41.82604, -87.71436 ...",0.006035
2,ARMOUR SQUARE,34,11,13635,2.776620e+07,"MULTIPOLYGON (((-87.62917 41.84556, -87.62947 ...",0.552168
3,ASHBURN,70,4,43283,1.354603e+08,"MULTIPOLYGON (((-87.71255 41.75734, -87.71252 ...",0.022850
4,AUBURN GRESHAM,71,23,45770,1.050654e+08,"MULTIPOLYGON (((-87.63990 41.75615, -87.63990 ...",0.296264


In [15]:
# Initialize the map:
groceries_by_community = folium.Map([41.84, -87.7099], zoom_start=10.5)
         
# Add the color for the chloropleth:
groceries_by_community.choropleth(
     geo_data=fp,
     name='Grocery Stores Density',
     data=groceries_intensity,
     columns=['Community Area', 'markets_by_surface'],
     key_on='feature.properties.community',
     fill_color='YlGnBu', fill_opacity=0.7, line_opacity=0.3,
     legend_name='Grocery Stores Density Per Area',
     highlight=True,
)   

# This is used to be able to render the tooltip -> folium is not really flexible in that sense
x = boundaries_community_areas.merge(groceries_intensity,\
                              left_on=['area_number','community','geometry','shape_area'], \
                                     right_on=['area_number','Community Area','geometry','shape_area'])

folium.GeoJson(
    x,
    name='community',
    show=True,
    style_function=lambda x: {
        'fillColor': 'blue',
        'color': 'black',
        'weight': 1,
        'fillOpacity':0
    },
    highlight_function=lambda x: {
        'fillOpacity':1
    },
    tooltip=folium.features.GeoJsonTooltip(
        fields=['community', 'nbr_markets','Population'],
        aliases=['Community Area', 'Number of Grocery Stores','Population Count'],
    ),
).add_to(groceries_by_community)

# Save to html
groceries_by_community.save('./groceries_by_community.html')
#groceries_by_community

**The map can be accessed [here](https://aaag97.github.io/ada-2019-project-databusters/groceries_by_community.html)**

"Chicago’s segregation is certainly legendary, with the North and South sides divided by class and race. To keep it stereotypically simple: The North Side is white, the South Side is black." [source](https://chicago.eater.com/2018/12/13/18138387/chicago-magazine-john-kessler-food-scene-racism-immigration-food)

We certainly see a difference in the concentration of grocery stores between the North and the South, the south being quite sparse.

* **For the next milestone, we will visualize the evolution of this distribution accross years using timestamps.**
* **We will also plot the density of markets per 1'000 habitants in a community area which seems more fair than plotting the absolute number of markets.**
* **We also have in our dataset health related establishments serving food, and from the health indicators dataset, diabetes and other diet related diseases rates which can be interesting to explore as well against what get from the markets visualization.**

## Diet-related deaths

### Diabetes related deaths (US baseline 73.1 !)

In [11]:
socioeconomic_factors.head()

,Community Area,Community Area Name,Colorectal Cancer,Diabetes-related,Stroke (Cerebrovascular Disease),Below Poverty Level,Per Capita Income,Unemployment,Population
0,1,Rogers Park,25.3,77.1,33.7,22.7,23714,7.5,55500
1,2,West Ridge,17.3,60.5,34.7,15.1,21375,7.9,73723
2,3,Uptown,20.5,80.0,41.7,22.7,32355,7.7,56296
3,4,Lincoln Square,8.6,55.4,36.9,9.5,35503,6.8,41483
4,5,North Center,26.1,49.8,41.6,7.1,51615,4.5,34942


In [46]:
# Initialize the map:
diabetes_map = folium.Map([41.84, -87.7099], zoom_start=10.5)
         
# Add the color for the chloropleth:
diabetes_map.choropleth(
     geo_data=fp,
     name='Diabetes_related_mortality',
     data=socioeconomic_factors,
     columns=['Community Area', 'Diabetes-related'],
     key_on='feature.properties.area_number',
     fill_color='YlOrBr', fill_opacity=0.7, line_opacity=0.3,
     legend_name='Diabetes related mortality per 100,000 persons',
     highlight=True,
)   

# This is used to be able to render the tooltip -> folium is not really flexible in that sense
x = boundaries_community_areas.merge(socioeconomic_factors,\
                              left_on=['area_number'], \
                                     right_on=['Community Area'])

folium.GeoJson(
    x,
    name='community',
    show=True,
    style_function=lambda x: {
        'fillColor': 'red',
        'color': 'black',
        'weight': 1,
        'fillOpacity':0
    },
    highlight_function=lambda x: {
        'fillOpacity':1
    },
    tooltip=folium.features.GeoJsonTooltip(
        fields=['community','Diabetes-related'],
        aliases=['Community Area','Number of Diabetes related deaths'],
    ),
).add_to(diabetes_map)

# Save to html
diabetes_map.save('./diabetes_mortality.html')

### Colorectal Cancer (US baseline 17)

In [47]:
# Initialize the map:
colorectal_map = folium.Map([41.84, -87.7099], zoom_start=10.5)
         
# Add the color for the chloropleth:
colorectal_map.choropleth(
     geo_data=fp,
     name='colorectal_cancer_related_mortality',
     data=socioeconomic_factors,
     columns=['Community Area', 'Colorectal Cancer'],
     key_on='feature.properties.area_number',
     fill_color='YlOrRd', fill_opacity=0.7, line_opacity=0.3,
     legend_name='Colorectal cancer related mortality per 100,000 persons',
     highlight=True,
)   

# This is used to be able to render the tooltip -> folium is not really flexible in that sense
x = boundaries_community_areas.merge(socioeconomic_factors,\
                              left_on=['area_number'], \
                                     right_on=['Community Area'])

folium.GeoJson(
    x,
    name='community',
    show=True,
    style_function=lambda x: {
        'fillColor': 'red',
        'color': 'black',
        'weight': 1,
        'fillOpacity':0
    },
    highlight_function=lambda x: {
        'fillOpacity':1
    },
    tooltip=folium.features.GeoJsonTooltip(
        fields=['community','Colorectal Cancer'],
        aliases=['Community Area','Number of Colorectal Cancer related deaths'],
    ),
).add_to(colorectal_map)

# Save to html
colorectal_map.save('./colorectal_map.html')

### Stroke (US baseline 42.2)

In [56]:
# Initialize the map:
stroke_map = folium.Map([41.84, -87.7099], zoom_start=10.5)
         
# Add the color for the chloropleth:
stroke_map.choropleth(
     geo_data=fp,
     name='stroke_mortality',
     data=socioeconomic_factors,
     columns=['Community Area', 'Stroke (Cerebrovascular Disease)'],
     key_on='feature.properties.area_number',
     fill_color='Reds', fill_opacity=0.7, line_opacity=0.3,
     legend_name='Stroke mortality per 100,000 persons',
     highlight=True,
)   

# This is used to be able to render the tooltip -> folium is not really flexible in that sense
x = boundaries_community_areas.merge(socioeconomic_factors,\
                              left_on=['area_number'], \
                                     right_on=['Community Area'])

folium.GeoJson(
    x,
    name='community',
    show=True,
    style_function=lambda x: {
        'fillColor': 'black',
        'color': 'black',
        'weight': 1,
        'fillOpacity':0
    },
    highlight_function=lambda x: {
        'fillOpacity':1
    },
    tooltip=folium.features.GeoJsonTooltip(
        fields=['community','Stroke (Cerebrovascular Disease)'],
        aliases=['Community Area','Deaths by stroke'],
    ),
).add_to(stroke_map)

# Save to html
stroke_map.save('./stroke_map.html')


## Community areas by ethnicity

https://www.chicagohealthatlas.org/indicators/non-hispanic-white